# Exploratory Data Analysis - Engagement Time Series

This notebook explores the synthetic engagement time series data to understand:
- Distribution of engagement metrics (views, likes, comments, shares)
- Differences between normal and fake engagement patterns
- Temporal patterns and anomalies
- Class balance and data quality


In [ ]:
import sys
from pathlib import Path

# add project root to path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# set plotting style
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except OSError:
    try:
        plt.style.use('seaborn-darkgrid')
    except OSError:
        plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

# import project modules
from src.data.simulate_timeseries import generate_dataset
from src.data.preprocess import load_and_preprocess


## 1. Generate and Load Data

First, we generate a synthetic dataset with normal and fake engagement patterns.


In [ ]:
# generate dataset
data_path = project_root / "data" / "raw" / "engagement_timeseries.parquet"

# generate if it doesn't exist
if not data_path.exists():
    print("Generating dataset...")
    generate_dataset(
        n_normal=100,
        n_fake=30,
        length_days=30,
        frequency="H",
        output_path=str(data_path),
        output_format="parquet",
        random_seed=42
    )
else:
    print(f"Loading existing dataset from {data_path}")

# load and preprocess
df = load_and_preprocess(
    file_path=str(data_path),
    target_timezone="UTC",
    resample_frequency="h",
    handle_missing=True,
    missing_method="forward",
    normalize=False
)

print(f"\nDataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nLabel distribution:")
print(df['label'].value_counts())
print(f"\nNumber of unique videos: {df['id'].nunique()}")


## 2. Data Overview

Basic statistics and data quality checks.


In [ ]:
# basic statistics by label
print("Summary statistics by label:\n")
print(df.groupby('label')[['views', 'likes', 'comments', 'shares']].describe())

# check for missing values
print("\nMissing values:")
print(df[['views', 'likes', 'comments', 'shares']].isna().sum())

# check data types
print("\nData types:")
print(df.dtypes)


## 3. Distribution Analysis - Histograms

Compare the distribution of engagement metrics between normal and fake patterns.


In [ ]:
# create histograms for each metric
metrics = ['views', 'likes', 'comments', 'shares']
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for idx, metric in enumerate(metrics):
    ax = axes[idx]
    
    # plot histograms for normal and fake
    normal_data = df[df['label'] == 'normal'][metric]
    fake_data = df[df['label'] == 'fake'][metric]
    
    ax.hist(normal_data, bins=50, alpha=0.6, label='Normal', color='blue', density=True)
    ax.hist(fake_data, bins=50, alpha=0.6, label='Fake', color='red', density=True)
    
    ax.set_xlabel(metric.capitalize(), fontsize=12)
    ax.set_ylabel('Density', fontsize=12)
    ax.set_title(f'Distribution of {metric.capitalize()}', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# print statistics
print("\nMean values by label:")
print(df.groupby('label')[metrics].mean())
print("\nMedian values by label:")
print(df.groupby('label')[metrics].median())


## 4. Average Temporal Patterns

Compare average engagement curves over time for normal vs fake patterns.


In [ ]:
# normalize time for each video (0 to 1)
df_normalized_time = df.copy()
df_normalized_time['time_normalized'] = df_normalized_time.groupby('id')['timestamp'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() > x.min() else 0
)

# create time bins for averaging
n_bins = 50
df_normalized_time['time_bin'] = pd.cut(df_normalized_time['time_normalized'], bins=n_bins, labels=False)

# calculate average curves by label
avg_curves = df_normalized_time.groupby(['label', 'time_bin'])[metrics].mean().reset_index()

# plot average curves
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for idx, metric in enumerate(metrics):
    ax = axes[idx]
    
    normal_curve = avg_curves[avg_curves['label'] == 'normal']
    fake_curve = avg_curves[avg_curves['label'] == 'fake']
    
    ax.plot(normal_curve['time_bin'], normal_curve[metric], 
            label='Normal', linewidth=2, color='blue', marker='o', markersize=4)
    ax.plot(fake_curve['time_bin'], fake_curve[metric], 
            label='Fake', linewidth=2, color='red', marker='s', markersize=4)
    
    ax.set_xlabel('Normalized Time (0 = start, 1 = end)', fontsize=12)
    ax.set_ylabel(f'Average {metric.capitalize()}', fontsize=12)
    ax.set_title(f'Average {metric.capitalize()} Over Time', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 5. Data Overview

Basic statistics and data quality checks.


In [ ]:
# define metrics
metrics = ['views', 'likes', 'comments', 'shares']

# basic statistics by label
is_fake_col = df.get('is_fake_series', None)
if is_fake_col is None:
    is_fake_col = (df.get('label', pd.Series(['normal'] * len(df))) == 'fake')
df['label'] = is_fake_col.map({True: 'fake', False: 'normal'})

print("Summary statistics by label:\n")
print(df.groupby('label')[metrics].describe())

# check for missing values
print("\nMissing values:")
print(df[metrics].isna().sum())

# check data types
print("\nData types:")
print(df[metrics + ['timestamp']].dtypes)


In [ ]:
# select example videos
normal_ids = df[df['label'] == 'normal']['id'].unique()[:3]
fake_ids = df[df['label'] == 'fake']['id'].unique()[:3]

fig, axes = plt.subplots(3, 2, figsize=(16, 12))

# plot normal examples
for idx, video_id in enumerate(normal_ids):
    ax = axes[idx, 0]
    video_data = df[df['id'] == video_id].sort_values('timestamp')
    
    ax.plot(video_data['timestamp'], video_data['views'], 
            label='Views', linewidth=2, color='blue')
    ax.plot(video_data['timestamp'], video_data['likes'] * 20, 
            label='Likes (x20)', linewidth=2, color='green')
    ax.plot(video_data['timestamp'], video_data['comments'] * 50, 
            label='Comments (x50)', linewidth=2, color='orange')
    
    ax.set_title(f'Normal Video: {video_id}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Timestamp', fontsize=10)
    ax.set_ylabel('Engagement', fontsize=10)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

# plot fake examples
for idx, video_id in enumerate(fake_ids):
    ax = axes[idx, 1]
    video_data = df[df['id'] == video_id].sort_values('timestamp')
    
    ax.plot(video_data['timestamp'], video_data['views'], 
            label='Views', linewidth=2, color='blue')
    ax.plot(video_data['timestamp'], video_data['likes'] * 20, 
            label='Likes (x20)', linewidth=2, color='green')
    ax.plot(video_data['timestamp'], video_data['comments'] * 50, 
            label='Comments (x50)', linewidth=2, color='orange')
    
    ax.set_title(f'Fake Video: {video_id}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Timestamp', fontsize=10)
    ax.set_ylabel('Engagement', fontsize=10)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


## 6. Pattern Distinctness Analysis

Analyze how distinct fake patterns are from normal patterns using statistical measures.


## 7. Engagement Metrics Comparison Heatmap

Compare mean engagement metrics between normal and fake patterns using a heatmap.


In [ ]:
# create comparison heatmap
comparison_data = df.groupby('label')[metrics].agg(['mean', 'std', 'max', 'min']).T
comparison_data.columns = [f"{col[0]}_{col[1]}" for col in comparison_data.columns]

# create mean values heatmap
mean_comparison = df.groupby('label')[metrics].mean().T
mean_comparison.columns = ['Normal', 'Fake']

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# absolute values
sns.heatmap(mean_comparison, annot=True, fmt='.0f', cmap='YlOrRd', 
            cbar_kws={'label': 'Mean Value'}, ax=axes[0], linewidths=0.5)
axes[0].set_title('Mean Engagement Metrics by Label', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Metric', fontsize=12)

# normalized (fake/normal ratio)
ratio_comparison = (mean_comparison['Fake'] / (mean_comparison['Normal'] + 1e-6)).to_frame('Fake/Normal Ratio')
sns.heatmap(ratio_comparison, annot=True, fmt='.2f', cmap='RdYlGn_r', 
            center=1, vmin=0.5, vmax=2, cbar_kws={'label': 'Ratio'}, 
            ax=axes[1], linewidths=0.5)
axes[1].set_title('Fake/Normal Ratio by Metric', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Metric', fontsize=12)

plt.tight_layout()
plt.show()

print("Mean values comparison:")
print(mean_comparison)
print("\nFake/Normal ratios:")
print(ratio_comparison)


In [ ]:
# calculate per-video statistics
video_stats = df.groupby(['id', 'label']).agg({
    'views': ['mean', 'std', 'max', 'min'],
    'likes': ['mean', 'std', 'max', 'min'],
    'comments': ['mean', 'std', 'max', 'min'],
    'shares': ['mean', 'std', 'max', 'min']
}).reset_index()

video_stats.columns = ['_'.join(col).strip('_') if col[1] else col[0] for col in video_stats.columns.values]

# calculate coefficient of variation (std/mean) as a measure of variability
for metric in metrics:
    mean_col = f'{metric}_mean'
    std_col = f'{metric}_std'
    cv_col = f'{metric}_cv'
    video_stats[cv_col] = video_stats[std_col] / (video_stats[mean_col] + 1e-6)

# compare statistics between normal and fake
print("Per-video statistics comparison:\n")
comparison_cols = [col for col in video_stats.columns if any(m in col for m in metrics) and ('mean' in col or 'cv' in col)]
print(video_stats.groupby('label')[comparison_cols].mean())

# plot comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for idx, metric in enumerate(metrics):
    ax = axes[idx]
    
    normal_cv = video_stats[video_stats['label'] == 'normal'][f'{metric}_cv']
    fake_cv = video_stats[video_stats['label'] == 'fake'][f'{metric}_cv']
    
    ax.boxplot([normal_cv.dropna(), fake_cv.dropna()], 
               labels=['Normal', 'Fake'])
    ax.set_ylabel('Coefficient of Variation', fontsize=12)
    ax.set_title(f'{metric.capitalize()} Variability (CV)', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 8. Correlation Heatmaps - Normal vs Fake

Compare correlation patterns between engagement metrics for normal and fake patterns.


In [ ]:
# compute correlation matrices
normal_df = df[df['label'] == 'normal'][metrics]
fake_df = df[df['label'] == 'fake'][metrics]

normal_corr = normal_df.corr()
fake_corr = fake_df.corr()

# plot correlation heatmaps side by side
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# normal correlation heatmap
sns.heatmap(normal_corr, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, vmin=-1, vmax=1, square=True, ax=axes[0],
            cbar_kws={'label': 'Correlation'})
axes[0].set_title('Normal Engagement - Correlation Matrix', fontsize=14, fontweight='bold')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha='right')
axes[0].set_yticklabels(axes[0].get_yticklabels(), rotation=0)

# fake correlation heatmap
sns.heatmap(fake_corr, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, vmin=-1, vmax=1, square=True, ax=axes[1],
            cbar_kws={'label': 'Correlation'})
axes[1].set_title('Fake Engagement - Correlation Matrix', fontsize=14, fontweight='bold')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45, ha='right')
axes[1].set_yticklabels(axes[1].get_yticklabels(), rotation=0)

plt.tight_layout()
plt.show()

# difference heatmap
corr_diff = fake_corr - normal_corr
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
sns.heatmap(corr_diff, annot=True, fmt='.2f', cmap='RdBu_r', 
            center=0, vmin=-0.5, vmax=0.5, square=True, ax=ax,
            cbar_kws={'label': 'Difference (Fake - Normal)'})
ax.set_title('Correlation Difference: Fake - Normal', fontsize=14, fontweight='bold')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
plt.tight_layout()
plt.show()

print("Correlation differences (Fake - Normal):")
print(corr_diff.round(3))


## 7. Class Balance Analysis

Check if the dataset is balanced and suitable for training.


In [ ]:
# class distribution
label_counts = df['label'].value_counts()
label_proportions = df['label'].value_counts(normalize=True)

print("Class distribution:")
print(label_counts)
print("\nClass proportions:")
print(label_proportions)

# plot class distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# count plot
axes[0].bar(label_counts.index, label_counts.values, color=['blue', 'red'], alpha=0.7)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_xlabel('Label', fontsize=12)
axes[0].set_title('Class Distribution (Count)', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# add count labels on bars
for i, v in enumerate(label_counts.values):
    axes[0].text(i, v + max(label_counts.values) * 0.01, str(v), 
                 ha='center', va='bottom', fontweight='bold')

# proportion pie chart
axes[1].pie(label_proportions.values, labels=label_proportions.index, 
            autopct='%1.1f%%', startangle=90, colors=['blue', 'red'])
axes[1].set_title('Class Distribution (Proportion)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# videos per class
videos_per_class = df.groupby('label')['id'].nunique()
print("\nNumber of unique videos per class:")
print(videos_per_class)
print(f"\nRatio (normal/fake): {videos_per_class['normal'] / videos_per_class['fake']:.2f}")

# check if balanced (within reasonable range)
balance_ratio = label_proportions['normal'] / label_proportions['fake']
print(f"\nBalance ratio: {balance_ratio:.2f}")
if 0.5 <= balance_ratio <= 2.0:
    print("Dataset is reasonably balanced for training.")
else:
    print("Warning: Dataset may be imbalanced. Consider using class weights or resampling.")


## 8. Correlation Analysis

Examine correlations between different engagement metrics for normal vs fake patterns.


In [ ]:
# calculate correlation matrices
normal_df = df[df['label'] == 'normal'][metrics]
fake_df = df[df['label'] == 'fake'][metrics]

normal_corr = normal_df.corr()
fake_corr = fake_df.corr()

# plot correlation matrices
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.heatmap(normal_corr, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, vmin=-1, vmax=1, ax=axes[0], square=True)
axes[0].set_title('Normal Engagement - Correlation Matrix', fontsize=14, fontweight='bold')

sns.heatmap(fake_corr, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, vmin=-1, vmax=1, ax=axes[1], square=True)
axes[1].set_title('Fake Engagement - Correlation Matrix', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# compare correlations
print("Correlation differences (Fake - Normal):")
corr_diff = fake_corr - normal_corr
print(corr_diff)


## 9. Summary and Conclusions

Key findings from the exploratory analysis.


In [ ]:
print("=" * 60)
print("EDA SUMMARY")
print("=" * 60)

print(f"\n1. Dataset size: {len(df):,} rows, {df['id'].nunique()} unique videos")
print(f"   - Normal: {label_counts['normal']:,} rows ({label_proportions['normal']:.1%})")
print(f"   - Fake: {label_counts['fake']:,} rows ({label_proportions['fake']:.1%})")

print(f"\n2. Class balance:")
print(f"   - Ratio (normal/fake): {balance_ratio:.2f}")
if 0.5 <= balance_ratio <= 2.0:
    print("   - Status: Reasonably balanced")
else:
    print("   - Status: Imbalanced - consider class weights")

print(f"\n3. Pattern distinctness:")
for metric in metrics:
    normal_mean = df[df['label'] == 'normal'][metric].mean()
    fake_mean = df[df['label'] == 'fake'][metric].mean()
    diff_pct = ((fake_mean - normal_mean) / normal_mean) * 100
    print(f"   - {metric.capitalize()}: {diff_pct:+.1f}% difference")

print(f"\n4. Data quality:")
print(f"   - Missing values: {df[metrics].isna().sum().sum()}")
print(f"   - Negative values: {(df[metrics] < 0).sum().sum()}")

print(f"\n5. Conclusion:")
print("   - Dataset is ready for feature engineering and modeling")
print("   - Clear visual differences between normal and fake patterns")
print("   - Temporal patterns show distinct behaviors")
print("=" * 60)
